In [ ]:

from transformers import AutoModel, AutoTokenizer

model_name = "TheBloke/WizardLM-7B-uncensored-GPTQ"



In [ ]:
# import transformers
# from transformers import AutoConfig, AutoModelForCausalLM
# import torch
# import time
# import logging
#
# logger = logging.getLogger("test")
#
# def load_quant(model,
#                checkpoint,
#                wbits,
#                groupsize=-1,
#                faster_kernel=False,
#                exclude_layers=None,
#                kernel_switch_threshold=128,
#                eval=True):
#     exclude_layers = exclude_layers or ['lm_head']
#
#     def noop(*args, **kwargs):
#         pass
#
#     config = AutoConfig.from_pretrained(model, trust_remote_code=False)
#     torch.nn.init.kaiming_uniform_ = noop
#     torch.nn.init.uniform_ = noop
#     torch.nn.init.normal_ = noop
#
#     torch.set_default_dtype(torch.half)
#     transformers.modeling_utils._init_weights = False
#     torch.set_default_dtype(torch.half)
#     model = AutoModelForCausalLM.from_config(config, trust_remote_code=False)
#     torch.set_default_dtype(torch.float)
#     if eval:
#         model = model.eval()
#
#     # layers = find_layers(model)
#     # for name in exclude_layers:
#     #     if name in layers:
#     #         del layers[name]
#     #
#     # if not is_triton:
#     #     gptq_args = inspect.getfullargspec(make_quant).args
#     #
#     #     make_quant_kwargs = {
#     #         'module': model,
#     #         'names': layers,
#     #         'bits': wbits,
#     #     }
#     #     if 'groupsize' in gptq_args:
#     #         make_quant_kwargs['groupsize'] = groupsize
#     #     if 'faster' in gptq_args:
#     #         make_quant_kwargs['faster'] = faster_kernel
#     #     if 'kernel_switch_threshold' in gptq_args:
#     #         make_quant_kwargs['kernel_switch_threshold'] = kernel_switch_threshold
#     #
#     #     make_quant(**make_quant_kwargs)
#     # else:
#     #     quant.make_quant_linear(model, layers, wbits, groupsize)
#     #
#     # del layers
#     # if checkpoint.endswith('.safetensors'):
#     #     from safetensors.torch import load_file as safe_load
#     #     model.load_state_dict(safe_load(checkpoint), strict=False)
#     # else:
#     #     model.load_state_dict(torch.load(checkpoint), strict=False)
#     #
#     # if is_triton:
#     #     if shared.args.quant_attn:
#     #         quant.make_quant_attn(model)
#     #
#     #     if eval and shared.args.fused_mlp:
#     #         quant.make_fused_mlp(model)
#     #
#     #     if shared.args.warmup_autotune:
#     #         quant.autotune_warmup_linear(model, transpose=not eval)
#     #         if eval and shared.args.fused_mlp:
#     #             quant.autotune_warmup_fused(model)
#     #
#     # model.seqlen = 2048
#     # return model
#
# def load_quantized(model_name):
#     # if shared.args.model_type is None:
#     #     logger.error("The model could not be loaded because its type could not be inferred from its name.")
#     #     logger.error("Please specify the type manually using the --model_type argument.")
#     #     return None
#
#     # Select the appropriate load_quant function
#     # model_type = shared.args.model_type.lower()
#     # if shared.args.pre_layer and model_type == 'llama':
#     #     load_quant = llama_inference_offload.load_quant
#     # elif model_type in ('llama', 'opt', 'gptj'):
#     #     if shared.args.pre_layer:
#     #         logger.warning("Ignoring --pre_layer because it only works for llama model type.")
#     #
#     #     load_quant = _load_quant
#     # else:
#     #     logger.error("Unknown pre-quantized model type specified. Only 'llama', 'opt' and 'gptj' are supported")
#     #     exit()
#
#     # Find the quantized model weights file (.pt/.safetensors)
#     path_to_model = Path(f'{shared.args.model_dir}/{model_name}')
#     pt_path = find_quantized_model_file(model_name)
#     if not pt_path:
#         logger.error("Could not find the quantized model in .pt or .safetensors format, exiting...")
#         exit()
#     else:
#         logger.info(f"Found the following quantized model: {pt_path}")
#
#     # # qwopqwop200's offload
#     # if model_type == 'llama' and shared.args.pre_layer:
#     #     if len(shared.args.pre_layer) == 1:
#     #         pre_layer = shared.args.pre_layer[0]
#     #     else:
#     #         pre_layer = shared.args.pre_layer
#
#     model = load_quant(str(path_to_model), str(pt_path))
#     # else:
#     #     threshold = False if model_type == 'gptj' else 128
#     #     model = load_quant(str(path_to_model), str(pt_path), shared.args.wbits, shared.args.groupsize, kernel_switch_threshold=threshold)
#     #
#     #     # accelerate offload (doesn't work properly)
#     #     if shared.args.gpu_memory or torch.cuda.device_count() > 1:
#     #         if shared.args.gpu_memory:
#     #             memory_map = list(map(lambda x: x.strip(), shared.args.gpu_memory))
#     #             max_cpu_memory = shared.args.cpu_memory.strip() if shared.args.cpu_memory is not None else '99GiB'
#     #             max_memory = {}
#     #             for i in range(len(memory_map)):
#     #                 max_memory[i] = f'{memory_map[i]}GiB' if not re.match('.*ib$', memory_map[i].lower()) else memory_map[i]
#     #
#     #             max_memory['cpu'] = f'{max_cpu_memory}GiB' if not re.match('.*ib$', max_cpu_memory.lower()) else max_cpu_memory
#     #         else:
#     #             max_memory = accelerate.utils.get_balanced_memory(model)
#     #
#     #         device_map = accelerate.infer_auto_device_map(model, max_memory=max_memory, no_split_module_classes=["LlamaDecoderLayer"])
#     #         logger.info("Using the following device map for the quantized model:", device_map)
#     #         # https://huggingface.co/docs/accelerate/package_reference/big_modeling#accelerate.dispatch_model
#     #         model = accelerate.dispatch_model(model, device_map=device_map, offload_buffers=True)
#     #
#     #     # No offload
#     #     elif not shared.args.cpu:
#     #         model = model.to(torch.device('cuda:0'))
#
#     return model
#
# def load_model(model_name, loader=None):
#     logger.info(f"Loading {model_name}...")
#     t0 = time.time()
#
#     # loader GPTQ
#     model = load_quantized(model_name)
#
#     tokenizer = "todo"
#
#     # shared.is_seq2seq = False
#     # load_func_map = {
#     #     'Transformers': huggingface_loader,
#     #     'AutoGPTQ': AutoGPTQ_loader,
#     #     'GPTQ-for-LLaMa': GPTQ_loader,
#     #     'llama.cpp': llamacpp_loader,
#     #     'FlexGen': flexgen_loader,
#     #     'RWKV': RWKV_loader,
#     #     'ExLlama': ExLlama_loader,
#     #     'ExLlama_HF': ExLlama_HF_loader
#     # }
#     #
#     # if loader is None:
#     #     if shared.args.loader is not None:
#     #         loader = shared.args.loader
#     #     else:
#     #         loader = infer_loader(model_name)
#     #         if loader is None:
#     #             logger.error('The path to the model does not exist. Exiting.')
#     #             return None, None
#     #
#     # shared.args.loader = loader
#     # output = load_func_map[loader](model_name)
#     # if type(output) is tuple:
#     #     model, tokenizer = output
#     # else:
#     #     model = output
#     #     if model is None:
#     #         return None, None
#     #     else:
#     #         tokenizer = load_tokenizer(model_name, model)
#
#     # Hijack attention with xformers
#     if any((shared.args.xformers, shared.args.sdp_attention)):
#         llama_attn_hijack.hijack_llama_attention()
#
#     logger.info(f"Loaded the model in {(time.time()-t0):.2f} seconds.\n")
#     return model, tokenizer

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

In [2]:
import re
import time
import torch
import logging
from pathlib import Path
import modules.shared as shared
import hashlib

print(shared.args.cpu)

logger = logging.getLogger("test")

def AutoGPTQ_loader(model_name):
    import modules.AutoGPTQ_loader

    return modules.AutoGPTQ_loader.load_quantized(model_name)

def load_tokenizer(model_name, model):
    tokenizer = None
    path_to_model = Path(f"{shared.args.model_dir}/{model_name}/")
    if any(s in model_name.lower() for s in ['gpt-4chan', 'gpt4chan']) and Path(f"{shared.args.model_dir}/gpt-j-6B/").exists():
        tokenizer = AutoTokenizer.from_pretrained(Path(f"{shared.args.model_dir}/gpt-j-6B/"))
    elif path_to_model.exists():
        tokenizer = AutoTokenizer.from_pretrained(
            path_to_model,
            trust_remote_code=shared.args.trust_remote_code,
            use_fast=False
        )

    if tokenizer.__class__.__name__ == 'LlamaTokenizer':
        pairs = [
            ['tokenizer_config.json', '516c6167c884793a738c440e29ccb80c15e1493ffc965affc69a1a8ddef4572a'],
            ['special_tokens_map.json', 'ff3b4a612c4e447acb02d40071bddd989fe0da87eb5b7fe0dbadfc4f74de7531']
        ]

        for pair in pairs:
            p = path_to_model / pair[0]
            if p.exists():
                with open(p, "rb") as f:
                    bytes = f.read()

                file_hash = hashlib.sha256(bytes).hexdigest()
                if file_hash != pair[1]:
                    logger.warning(f"{p} is different from the original LlamaTokenizer file. It is either customized or outdated.")

    return tokenizer


def get_model_settings_from_yamls(model):
    settings = shared.model_config
    model_settings = {}
    for pat in settings:
        if re.match(pat.lower(), model.lower()):
            for k in settings[pat]:
                model_settings[k] = settings[pat][k]

    return model_settings

def infer_loader(model_name):
    path_to_model = Path(f'{shared.args.model_dir}/{model_name}')
    model_settings = get_model_settings_from_yamls(model_name)
    if not path_to_model.exists():
        loader = None
    elif Path(f'{shared.args.model_dir}/{model_name}/quantize_config.json').exists() or ('wbits' in model_settings and type(model_settings['wbits']) is int and model_settings['wbits'] > 0):
        loader = 'AutoGPTQ'
    elif len(list(path_to_model.glob('*ggml*.bin'))) > 0:
        loader = 'llama.cpp'
    elif re.match('.*ggml.*\.bin', model_name.lower()):
        loader = 'llama.cpp'
    elif re.match('.*rwkv.*\.pth', model_name.lower()):
        loader = 'RWKV'
    elif shared.args.flexgen:
        loader = 'FlexGen'
    else:
        loader = 'Transformers'

    return loader


def load_model(model_name, loader=None):
    logger.info(f"Loading {model_name}...")
    t0 = time.time()

    shared.is_seq2seq = False

    # using AutoGPTQ

    load_func_map = {
        # 'Transformers': huggingface_loader,
        'AutoGPTQ': AutoGPTQ_loader,
        # 'GPTQ-for-LLaMa': GPTQ_loader,
        # 'llama.cpp': llamacpp_loader,
        # 'FlexGen': flexgen_loader,
        # 'RWKV': RWKV_loader,
        # 'ExLlama': ExLlama_loader,
        # 'ExLlama_HF': ExLlama_HF_loader
    }

    if loader is None:
        if shared.args.loader is not None:
            loader = shared.args.loader
        else:
            loader = infer_loader(model_name)
            if loader is None:
                logger.error('The path to the model does not exist. Exiting.')
                return None, None

    shared.args.loader = loader
    output = load_func_map[loader](model_name)
    if type(output) is tuple:
        model, tokenizer = output
    else:
        model = output
        if model is None:
            return None, None
        else:
            tokenizer = load_tokenizer(model_name, model)

    # Hijack attention with xformers
    if any((shared.args.xformers, shared.args.sdp_attention)):
        llama_attn_hijack.hijack_llama_attention()

    logger.info(f"Loaded the model in {(time.time()-t0):.2f} seconds.\n")
    return model, tokenizer


question = "what color is the sky?"

model, tokenizer = load_model("7bwizard")
tokenizer = load_tokenizer("7bwizard", None)
tokens = tokenizer.encode(question, add_special_tokens=True)
input_ids = torch.tensor([tokens])
print(input_ids)

False


TypeError: exceptions must derive from BaseException

GPTQ TEST